# 소프트맥스 (다중 분류 로지스틱 회귀 모델)
M개의 입력을 받아 N개의 클래스로 출력하는 로지스틱 회귀 모델을 케라스로 구현해보도록 하겠습니다.  
보통 다중 분류 로지스틱 회귀 모델을 소프트맥스(softmax)라고 부릅니다.  
케라스에서 제공하는 MNIST 데이터를 사용하여 숫자를 0에서부터 9까지 분류해보도록 하겠습니다.  

In [1]:
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.utils import to_categorical
from keras.datasets import mnist

Using TensorFlow backend.


MNIST 손글씨 데이터를 다운로드 받아서 변수에 저장합니다.

In [2]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()

손글씨 데이터(X_train, X_test)가 가로 28픽셀, 세로 28픽셀로 구성된 것을 확인할 수 있습니다.  
학습에 사용될 X_train은 총 60000개의 데이터, 테스트에 사용될 X_test는 총 10000개의 데이터가 있습니다.

In [3]:
print("train data (count, row, column) : " + str(X_train.shape) )
print("test data  (count, row, column) : " + str(X_test.shape) )

train data (count, row, column) : (60000, 28, 28)
test data  (count, row, column) : (10000, 28, 28)


학습 데이터의 하나를 샘플로 보도록 하겠습니다. 아래 보시는 것처럼, 각각의 픽셀은 0부터 255까지의 값을 가지고 있습니다.

In [4]:
print(X_train[0])

[[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   3  18  18  18 126 136
  175  26 166 255 247 127   0   0   0   0]
 [  0   0   0   0   0   0   0   0  30  36  94 154 170 253 253 253 253 253
  225 172 253 242 195  64   0   0   0   0]
 [  0   0   0   0   0   0   0  49 238 253 253 253 253 253 253 253 253 251
   93  82  82  56  39   0   0   0   0   0]
 [  0   0   0   0   0   0   0  18 219 253 253 253 253 253 198 18

모델 학습 시작에 앞서, 데이터를 정규화합니다.  
정규화는 입력값을 0부터 1의 값으로 변경하게 됩니다.  
정규화된 입력값은 경사하강법으로 모델 학습 시, 보다 쉽고 빠르게 최적의 W,B를 찾는 데 도움을 줍니다.

In [5]:
X_train = X_train.astype('float32') 
X_test = X_test.astype('float32') 
X_train /= 255 
X_test /= 255

아래의 명령어를 통해 정규화된 데이터를 확인할 수 있습니다.

In [6]:
print(X_train[0])

[[0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.    

y_train, y_test는 손글씨 데이터 (28*28 픽셀 데이터)에 해당하는 숫자를 나타냅니다.  
y_train은 총 6만개, y_test는 총 1만개의 숫자를 가지고 있습니다.

In [7]:
print("train target (count) : " + str(y_train.shape) )
print("test target  (count) : " + str(y_test.shape) )

train target (count) : (60000,)
test target  (count) : (10000,)


아래의 코드를 실행하여, y_train과 y_test에서 샘플로 숫자를 출력해봅니다.

In [8]:
print("sample from train : " + str(y_train[0]) )
print("sample from test : " + str(y_test[0]) )

sample from train : 5
sample from test : 7


이번 실습에서는 28*28 픽셀의 지역적인 정보를 사용하지 않고, 단순히 정규화된 입력값만을 가지고,  
숫자 분류를 할 것이기 때문에, 행과 열의 구분 없이, 단순히 784 길이의 배열로 데이터를 단순화시킵니다.

In [9]:
input_dim = 784 #28*28 
X_train = X_train.reshape(60000, input_dim) 
X_test = X_test.reshape(10000, input_dim) 

아래의 명령어를 실행하여, 현재 우리의 데이터가 2차원 데이터가 아닌 단순한 1차원 데이터로 변경된 것을 확인할 수 있습니다.

In [10]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(60000, 784)
(60000,)
(10000, 784)
(10000,)


소프트맥스가 어떻게 10개의 숫자를 구분하는 지가 이번 실습의 핵심입니다.  
소프트맥스는 정규화된 여러개의 로지스틱회귀로 구성되어 있으며,  
10개의 로지스틱회귀를 배열로 나타낼 경우, [L0, L1, L2, L3, L4, L5, L6, L7, L8, L9]으로 나타낼 수 있습니다.  
각 인덱스는 각각의 숫자를 의미합니다.  
로지스틱회귀이기 때문에, 각 L의 값은 0부터 1의 값을 가지고 있으며, 만약 출력값이 [0.9, 0.1, 0, 0, 0, 0, 0, 0, 0, 0]일 경우,  
가장 높은 확률을 가진 첫번째 인덱스, 즉 0이 소프트맥스의 출력값이라고 볼 수 있습니다.

학습 시, y값과의 cross entropy를 측정해야하므로, 아래의 코드를 실행하여 y를 one hot encoding으로 변환시켜줍니다.

In [11]:
num_classes = 10
y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)

아래 코드를 실행하여, 5였던 값이, one hot encoding으로 변환되어,  
클래스 갯수만큼의 길이를 갖는 벡터로 변경이 되었고, 5에 해당되는 인덱스의 값만 1인 것을 확인할 수 있습니다.

In [12]:
print(y_train[0])

[0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]


케라스의 Sequential()을 사용하여 간단하게 소프트맥스를 구현할 수 있습니다.  
총 784개(28*28)의 입력을 받아서, 10개의 시그모이드 값을 출력하는 모델을 아래의 코드를 실행하여 구현합니다.

In [13]:
model = Sequential() 
model.add(Dense(input_dim=input_dim, units = 10, activation='softmax'))

모델의 학습을 진행합니다.  
10개의 클래스로 분류할 것이기 때문에, categorical_crossentropy를 비용함수로 사용한 경사하강법으로 최적의 W와 biases를 학습합니다.

In [14]:
model.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy']) 
model.fit(X_train, y_train, batch_size=2048, epochs=100, verbose=1) 

Epoch 1/100
60000/60000 [==============================] - 1s 9us/step - loss: 2.2915 - acc: 0.1608
Epoch 2/100
60000/60000 [==============================] - 0s 8us/step - loss: 2.0073 - acc: 0.3549
Epoch 3/100
60000/60000 [==============================] - 0s 7us/step - loss: 1.7861 - acc: 0.5324
Epoch 4/100
60000/60000 [==============================] - 0s 8us/step - loss: 1.6093 - acc: 0.6270
Epoch 5/100
60000/60000 [==============================] - 0s 8us/step - loss: 1.4670 - acc: 0.6843
Epoch 6/100
60000/60000 [==============================] - 0s 8us/step - loss: 1.3514 - acc: 0.7229A: 0s - loss: 1.3897 - acc: 
Epoch 7/100
60000/60000 [==============================] - 0s 7us/step - loss: 1.2566 - acc: 0.7495
Epoch 8/100
60000/60000 [==============================] - 0s 7us/step - loss: 1.1779 - acc: 0.7679
Epoch 9/100
60000/60000 [==============================] - 0s 8us/step - loss: 1.1119 - acc: 0.7822
Epoch 10/100
60000/60000 [==============================] - 0s 8us/step 

60000/60000 [==============================] - 0s 8us/step - loss: 0.4681 - acc: 0.8796
Epoch 82/100
60000/60000 [==============================] - 0s 7us/step - loss: 0.4666 - acc: 0.8799
Epoch 83/100
60000/60000 [==============================] - 0s 7us/step - loss: 0.4651 - acc: 0.8802
Epoch 84/100
60000/60000 [==============================] - 0s 7us/step - loss: 0.4636 - acc: 0.8804
Epoch 85/100
60000/60000 [==============================] - 0s 8us/step - loss: 0.4621 - acc: 0.8807
Epoch 86/100
60000/60000 [==============================] - 0s 8us/step - loss: 0.4607 - acc: 0.8810
Epoch 87/100
60000/60000 [==============================] - 0s 8us/step - loss: 0.4593 - acc: 0.8812
Epoch 88/100
60000/60000 [==============================] - 0s 7us/step - loss: 0.4580 - acc: 0.8814
Epoch 89/100
60000/60000 [==============================] - 0s 8us/step - loss: 0.4566 - acc: 0.8818
Epoch 90/100
60000/60000 [==============================] - 0s 7us/step - loss: 0.4553 - acc: 0.8821
Epo

테스트를 진행하여, 정확도를 측정합니다.

In [15]:
score = model.evaluate(X_test, y_test) 
print('Test accuracy:', score[1])

10000/10000 [==============================] - 0s 39us/step
Test accuracy: 0.8912


아래의 코드를 실행하여, 소프트맥스 모델의 구조를 쉽게 시각화 할 수 있습니다.  
총 10개의 로지스틱회귀가 있고, 각 로지스틱회귀는 784개의 weight와 1개의 bias를 갖고 있기 때문에,  
총 7850 (785*10)개의 Param이 있는 것을 보실 수 있습니다.

In [16]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 10)                7850      
Total params: 7,850
Trainable params: 7,850
Non-trainable params: 0
_________________________________________________________________


첫번째 레이어에 존재하는 w1, w2,...,w784, b1, b2,..., b10은 아래의 명령어로 확인하실 수 있습니다.

In [17]:
model.layers[0].weights

[<tf.Variable 'dense_1/kernel:0' shape=(784, 10) dtype=float32_ref>,
 <tf.Variable 'dense_1/bias:0' shape=(10,) dtype=float32_ref>]